In [2]:
import polars as pl
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
from sklearn.preprocessing import StandardScaler

matplotlib.use("Qt5Agg")

In [6]:
def load_stacked(format: str):
    df = None
    for i in range(20):
        tes_i = pl.read_csv(format.format(i))
        id = pl.Series("id", np.full(tes_i.height, i))
        sub_id = pl.Series("sub_id", np.arange(tes_i.height))
        tes_i = tes_i.with_columns(id, sub_id).select(
                "id", "sub_id", "kernel_time", "transfer_time", "left_nz_size", "right_nz_size", "left_rank", "right_rank"
            )
        if df is None:
            df = tes_i
        else:
            df = df.vstack(tes_i)
    return df

def normalize(df: pl.DataFrame) -> pl.DataFrame:
    # Normalize kernel_time
    s = StandardScaler()
    s.fit(df['kernel_time'].to_numpy().reshape(-1, 1))
    new_kernel = s.transform(df['kernel_time'].to_numpy().reshape(-1, 1))
    df = df.with_columns(pl.Series("kernel_time_normalized", new_kernel.reshape(-1)))

    # Normalize transfer_time
    s = StandardScaler()
    s.fit(df['transfer_time'].to_numpy().reshape(-1, 1))
    new_transfer = s.transform(df['transfer_time'].to_numpy().reshape(-1, 1))
    df = df.with_columns(pl.Series("transfer_time_normalized", new_transfer.reshape(-1)))

    # filter out all the outliers
    df = df.filter((pl.col("kernel_time_normalized") < 3).and_(pl.col("kernel_time_normalized") > -3).and_(
        pl.col("transfer_time_normalized") < 3).and_(pl.col("transfer_time_normalized") > -3))
    return df

def plot_times_hist(df: pl.DataFrame, path_to_save: str | None = None):
    # plot the hist of kernel times and transfer times in the same row
    fig, axs = plt.subplots(1, 2, figsize=(20, 10))
    axs[0].hist(df["kernel_time"].to_numpy(), bins=100)
    axs[0].set_title("Kernel Time")
    axs[0].set_xlabel("Time (ns)")
    axs[0].set_ylabel("Frequency")
    axs[1].hist(df["transfer_time"].to_numpy(), bins=100)
    axs[1].set_title("Transfer Time")
    axs[1].set_xlabel("Time (ns)")
    axs[1].set_ylabel("Frequency")
    # axs[1].set_xscale("log")
    # axs[1].set_xticks(np.logspace(5, 6, num=10))
    if path_to_save is not None:
        fig.savefig(path_to_save)
    plt.show()

def plot_kernel_times_on_rank(df: pl.DataFrame, path_to_save: str | None = None):
    # plot kernel times wrt left_nz_size
    fig, ax = plt.subplots()
    X = df.select((pl.col('left_rank') + pl.col('right_rank')).alias('res_rank'), pl.col('kernel_time') / 1e3).group_by("res_rank").agg(pl.col("kernel_time")).sort("res_rank").to_numpy()
    # X2 = tes.select(pl.col('left_rank'), pl.col('transfer_time') / 1e3).group_by("left_rank").agg(pl.col("transfer_time")).sort("left_rank").to_numpy()
    # xaxis = [str(int(x)) for x in np.logspace(0, 14, num=15, base=2)]

    # boxplot
    # ax.violinplot(X[:, 1], X[:, 0], showmeans=True, showextrema=True)
    ax.boxplot(X[:, 1], positions=X[:, 0], showmeans=True, meanline=True, showfliers=False)
    # ax.boxplot(X2[:, 1], positions=X1[:, 0], showmeans=True, meanline=True, showfliers=False)

    ax.set_title("Kernel Time vs Resulting Rank")
    ax.set_xlabel("Resulting Rank")
    ax.set_ylabel("Kernel Time (us)")
    ax.grid()
    # ax.set_xscale("log", base=2)
    # ax.set_xticks(xaxis)
    if path_to_save is not None:
        fig.savefig(path_to_save)
    plt.show()

def plot_transfer_times_on_rank(df: pl.DataFrame, path_to_save: str | None = None):
    # plot kernel times wrt left_nz_size
    fig, ax = plt.subplots()
    X = df.select((pl.col('left_rank') + pl.col('right_rank')).alias('res_rank'), pl.col('transfer_time') / 1e3).group_by("res_rank").agg(pl.col("transfer_time")).sort("res_rank").to_numpy()
    # xaxis = [str(int(x)) for x in np.logspace(0, 14, num=15, base=2)]

    # boxplot
    # ax.violinplot(X[:, 1], X[:, 0], showmeans=True, showextrema=True)
    ax.boxplot(X[:, 1], positions=X[:, 0], showmeans=True, meanline=True, showfliers=False)

    ax.set_title("Kernel Time vs Resulting Rank")
    ax.set_xlabel("Resulting Rank")
    ax.set_ylabel("Kernel Time (us)")
    # ax.set_yscale("log")
    ax.grid()
    # ax.set_xscale("log", base=2)
    # ax.set_xticks(xaxis)
    if path_to_save is not None:
        fig.savefig(path_to_save)
    plt.show()

In [7]:
gv = load_stacked("../results/te_stats_golden-vectors-{}.csv")
gv.describe()

statistic,id,sub_id,kernel_time,transfer_time,left_nz_size,right_nz_size,left_rank,right_rank
str,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",2340.0,2340.0,2340.0,2340.0,2340.0,2340.0,2340.0,2340.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",9.5,58.0,8916.964103,365131.028632,3.435897,3.435897,1.512821,1.512821
"""std""",5.767514,33.780976,2991.673162,238560.640195,1.827932,1.827932,0.564223,0.564223
"""min""",0.0,0.0,4819.0,266571.0,2.0,2.0,1.0,1.0
"""25%""",5.0,29.0,7875.0,340060.0,2.0,2.0,1.0,1.0
"""50%""",10.0,58.0,8737.0,359857.0,4.0,4.0,1.0,1.0
"""75%""",14.0,87.0,9848.0,385795.0,4.0,4.0,2.0,2.0
"""max""",19.0,116.0,84338.0,1.1668845e7,16.0,16.0,3.0,3.0


In [8]:
ss = load_stacked("../results/te_stats_sparse_stress-{}.csv")
ss.describe()

statistic,id,sub_id,kernel_time,transfer_time,left_nz_size,right_nz_size,left_rank,right_rank
str,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",9.5,9.5,8029.025,781442.8425,103.3,103.3,3.25,3.25
"""std""",5.773503,5.773503,4905.999771,1.1537e6,243.689498,243.689498,3.034885,3.034885
"""min""",0.0,0.0,5180.0,272843.0,2.0,2.0,1.0,1.0
"""25%""",5.0,5.0,5952.0,316575.0,2.0,2.0,1.0,1.0
"""50%""",10.0,10.0,6652.0,425069.0,2.0,2.0,1.0,1.0
"""75%""",14.0,14.0,9027.0,795386.0,32.0,32.0,5.0,5.0
"""max""",19.0,19.0,82054.0,1.9830964e7,1024.0,1024.0,10.0,10.0


In [9]:
df = normalize(gv)
plot_times_hist(df, "hist_gv.png")
plot_kernel_times_on_rank(df, "kernel_time_vs_rank_gv.png")
plot_transfer_times_on_rank(df, "transfer_time_vs_rank_gv.png")

In [10]:
df = normalize(ss)
plot_times_hist(df, "hist_ss.png")
plot_kernel_times_on_rank(df, "kernel_time_vs_rank_ss.png")
plot_transfer_times_on_rank(df, "transfer_time_vs_rank_ss.png")

In [78]:
from scipy.optimize import curve_fit

X = df.select((pl.col('left_rank') + pl.col('right_rank')).alias('res_rank'), pl.col('transfer_time') / 1e3).group_by("res_rank").agg(pl.col("transfer_time").mean()).sort("res_rank").to_numpy()

# fit a linear model (composed of different exponential functions) to the data
def exp(x, a, b, c):
    return a * np.exp(b * x) + c

potp, pcov = curve_fit(exp, X[:, 0], X[:, 1])

y_fit = exp(X[:, 0], *potp)
y_err = np.sqrt(np.diag(pcov))
y_fit_up = exp(X[:, 0], *(potp + y_err))
y_fit_dw = exp(X[:, 0], *(potp - y_err))

# plot the data and the fitted curve
fig, ax = plt.subplots()
ax.plot(X[:, 0], X[:, 1], 'o', label='Original Data')
ax.plot(X[:, 0], y_fit, 'r-', label='Fitted Curve')
ax.fill_between(X[:, 0], y_fit_up, y_fit_dw, color='red', alpha=0.2)
ax.set_title("Transfer Time vs Resulting Rank Fitted Curve")
ax.set_xlabel("Resulting Rank")
ax.set_ylabel("Transfer Time (us)")
ax.legend()
ax.grid()
plt.show()

In [77]:
# make a projection using also the covariance matrix
new_x = np.linspace(0, 30, 100)

y_fit = exp(new_x, *potp)
y_err = np.sqrt(np.diag(pcov))
y_fit_up = exp(new_x, *(potp + y_err))
y_fit_dw = exp(new_x, *(potp - y_err))


fig, ax = plt.subplots()
ax.plot(X[:, 0], X[:, 1], 'o', label='Original Data')
ax.plot(new_x, y_fit, 'r-', label='Fitted Curve')
ax.fill_between(new_x, y_fit_up, y_fit_dw, color='red', alpha=0.2)

# add horizontal line at 1us, 1ms, 1s
ax.axhline(y=1e3, color='b', linestyle='--', label='1ms')
ax.axhline(y=1e6, color='y', linestyle='--', label='1s')
ax.axhline(y=1e6*60, color='r', linestyle='--', label='1m')

# set floating labels for resulting rank with those times
ax.text(5, 1e3*1.5, "1ms", color='b', fontsize=11, ha='right')
ax.text(5, 1e6*1.5, "1s", color='y', fontsize=11, ha='right')
ax.text(5, 1e6*60*1.5, "1m", color='r', fontsize=11, ha='right')

ax.set_title("Transfer Time vs Resulting Rank Projection")
ax.set_xlabel("Resulting Rank")
ax.set_ylabel("Transfer Time (us)")
ax.set_yscale("log")
ax.grid()
ax.legend()
plt.show()